In [1]:
import numpy as np
import pandas as pd


In [5]:
# Загрузим данные в пандас указав \t разделителем и назначив наименование столбцов
data = pd.read_csv("u.data", sep='\t', header=None, names=['user id' , 'item id' , 'rating' ,'timestamp'])
data.head()

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [12]:
# Рассчитаем среднюю оценку которую поставил каждый пользователь
grouped_data=data.groupby(['user id']).mean()
grouped_data[['rating']].head()

,rating
user id,
1,3.610294
2,3.709677
3,2.796296
4,4.333333
5,2.874286


In [15]:
# отсортируем филмы поколичеству оценок
grouped_data2=data.groupby(['item id']).count()
print(grouped_data2.sort_values(by='rating', ascending=False))

         user id  rating  timestamp
item id                            
50           583     583        583
258          509     509        509
100          508     508        508
181          507     507        507
294          485     485        485
286          481     481        481
288          478     478        478
1            452     452        452
300          431     431        431
121          429     429        429
174          420     420        420
127          413     413        413
56           394     394        394
7            392     392        392
98           390     390        390
237          384     384        384
117          378     378        378
172          367     367        367
222          365     365        365
313          350     350        350
204          350     350        350
405          344     344        344
79           336     336        336
210          331     331        331
151          326     326        326
173          324     324    

In [19]:
#Средствами pandasa соедините данные по оценкам с данными по пользователям (u.user)
u_data = pd.read_csv("u.user", sep='|', header=None, names=['user id' , 'age' , 'gender' ,'occupation', 'zip code'])
joined_data=u_data.set_index('user id').join(data.set_index('user id'))
joined_data.head()

,age,gender,occupation,zip code,item id,rating,timestamp
user id,,,,,,,
1,24,M,technician,85711,61,4,878542420
1,24,M,technician,85711,189,3,888732928
1,24,M,technician,85711,33,4,878542699
1,24,M,technician,85711,160,4,875072547
1,24,M,technician,85711,20,4,887431883


In [20]:
# Построить модель, которая предсказывает оценку для фильма и пользователя (воспользоваться RandomForestRegressor, тк мы решаем задачу регрессии)
# Подготовим наши данные
Model_data=joined_data.reset_index()
Model_data.head()

,user id,age,gender,occupation,zip code,item id,rating,timestamp
0,1,24,M,technician,85711,61,4,878542420
1,1,24,M,technician,85711,189,3,888732928
2,1,24,M,technician,85711,33,4,878542699
3,1,24,M,technician,85711,160,4,875072547
4,1,24,M,technician,85711,20,4,887431883


In [25]:
# переведем пол в цифровое значение
Model_data['gender_num']=Model_data['gender']
Model_data.loc[Model_data['gender'] == 'F', 'gender_num'] = 0
Model_data.loc[Model_data['gender'] == 'M', 'gender_num'] = 1
Model_data=Model_data.drop('gender', axis=1)
Model_data.head()

,user id,age,occupation,zip code,item id,rating,timestamp,gender_num
0,1,24,technician,85711,61,4,878542420,1
1,1,24,technician,85711,189,3,888732928,1
2,1,24,technician,85711,33,4,878542699,1
3,1,24,technician,85711,160,4,875072547,1
4,1,24,technician,85711,20,4,887431883,1


In [28]:
occup=Model_data['occupation'].unique()

In [30]:
# переведем позицию в числовое значение
Model_data['occupation_num']=Model_data['occupation']
n=0
for i in occup:
    Model_data.loc[Model_data['occupation'] == i, 'occupation_num'] = n
    n=n+1
Model_data.head()

,user id,age,occupation,zip code,item id,rating,timestamp,gender_num,occupation_num
0,1,24,technician,85711,61,4,878542420,1,0
1,1,24,technician,85711,189,3,888732928,1,0
2,1,24,technician,85711,33,4,878542699,1,0
3,1,24,technician,85711,160,4,875072547,1,0
4,1,24,technician,85711,20,4,887431883,1,0


In [32]:
Model_data=Model_data.drop('occupation', axis=1)
Model_data.head()

,user id,age,zip code,item id,rating,timestamp,gender_num,occupation_num
0,1,24,85711,61,4,878542420,1,0
1,1,24,85711,189,3,888732928,1,0
2,1,24,85711,33,4,878542699,1,0
3,1,24,85711,160,4,875072547,1,0
4,1,24,85711,20,4,887431883,1,0


In [33]:
# Построим матрицу корреляции
Model_data.corr()

,user id,age,item id,rating,timestamp,gender_num,occupation_num
user id,1.000000,-0.073599,0.010377,-0.009371,0.019082,-0.027605,-0.046332
age,-0.073599,1.000000,-0.012612,0.054460,0.156674,0.033358,0.193725
item id,0.010377,-0.012612,1.000000,-0.189124,0.041867,-0.034575,0.016416
rating,-0.009371,0.054460,-0.189124,1.000000,-0.012003,-0.000862,-0.016686
timestamp,0.019082,0.156674,0.041867,-0.012003,1.000000,-0.035490,0.072260
gender_num,-0.027605,0.033358,-0.034575,-0.000862,-0.035490,1.000000,0.005824
occupation_num,-0.046332,0.193725,0.016416,-0.016686,0.072260,0.005824,1.000000


In [ ]:
# Мы видим, что на оценку оказывает влияние фильм, а остальные параметры незначительно

In [37]:
# Построим модель для фильма и пользователя
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

Model_for_prediction=Model_data[['user id','item id']]
Model_results=Model_data[['rating']]

,rating
0,4
1,3
2,4
3,4
4,4


In [38]:
# Разделим данные на тестовые и обучающие
X_train, X_test, y_train, y_test = train_test_split(Model_for_prediction,Model_results,test_size = 0.2, random_state = 0)

print("Number train dataset: ", len(X_train))
print("Number test dataset: ", len(X_test))
print("Total number: ", len(X_train)+len(X_test))

Number train dataset:  80000
Number test dataset:  20000
Total number:  100000


In [40]:
model = RandomForestRegressor(n_estimators=100, max_features ='sqrt')
model.fit(X_train, y_train)

C:\Users\vova_oderii\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [43]:
model.feature_importances_

array([0.50634834, 0.49365166])

In [41]:
Y_pred = model.predict(X_test)

In [45]:
# Посчитаем квадратичную ошибку
from sklearn.metrics import mean_squared_error
print(mean_squared_error(Y_pred, np.array(y_test)))

1.164272935
